In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np


In [2]:
url = "https://www.betexplorer.com/football/africa/africa-cup-of-nations/fixtures/"

# Send a GET request to the URL
response = requests.get(url)

In [3]:

    # Parse the HTML content of the page
soup = BeautifulSoup(response.content, 'html.parser')

    # Find the elements containing the match data
match_elements = soup.select('.table-main tr')

In [4]:
# Find the table containing the match information
table = soup.find('table', {'class': 'table-main'})

In [5]:
matches_data = []
for row in table.find_all('tr')[1:]:  # Skip the header row
    columns = row.find_all('td')
    
    # Check if the row has enough columns
    if len(columns) >= 4:
        date = columns[0].text.strip()
        home_team = columns[1].text.strip()
        # away_team = columns[3].text.strip()

        matches_data.append([date, home_team])
    # else:
    #     print("Skipping row with insufficient columns:", row)


In [6]:
columns = ['Date', 'Team']
df = pd.DataFrame(matches_data, columns=columns)
df[['Home Team', 'Away Team']] = df['Team'].str.split(' - ', expand=True)
df = df.drop('Team', axis=1)

In [7]:
# df['Date'] = pd.to_datetime(df['Date'], format='%d.%m.%Y %H:%M', errors='coerce').dt.date

In [8]:
df['Date'] = pd.to_datetime(df['Date'], format='%d.%m.%Y %H:%M', errors='coerce').dt.strftime('%Y-%m-%d')

In [9]:
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d', errors='coerce')

In [10]:
# 

In [11]:
df.head()

,Date,Home Team,Away Team
0,2024-01-13,Ivory Coast,Guinea Bissau
1,2024-01-14,Nigeria,Equatorial Guinea
2,2024-01-18,Equatorial Guinea,Guinea Bissau
3,2024-01-18,Ivory Coast,Nigeria
4,2024-01-22,Equatorial Guinea,Ivory Coast


In [12]:
df = df.dropna(subset=['Date'])

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 0 to 34
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       30 non-null     datetime64[ns]
 1   Home Team  30 non-null     object        
 2   Away Team  30 non-null     object        
dtypes: datetime64[ns](1), object(2)
memory usage: 960.0+ bytes


In [14]:
# Assuming your DataFrame is named df
df.rename(columns={
    'Date': 'Date',
    'Home Team': 'home_team',
    'Away Team': 'away_team'
}, inplace=True)


In [15]:

df.to_csv('can_2023_matches.csv', index=False)


In [16]:
dt= pd.read_csv("International-football-results-20231121.csv")

In [17]:
dt.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False


In [18]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45315 entries, 0 to 45314
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   date        45315 non-null  object
 1   home_team   45315 non-null  object
 2   away_team   45315 non-null  object
 3   home_score  45315 non-null  int64 
 4   away_score  45315 non-null  int64 
 5   tournament  45315 non-null  object
 6   city        45315 non-null  object
 7   country     45315 non-null  object
 8   neutral     45315 non-null  bool  
dtypes: bool(1), int64(2), object(6)
memory usage: 2.8+ MB


In [19]:
# Rename the 'date' column to 'Date'
dt = dt.rename(columns={'date': 'Date'})


In [20]:

# Convert 'Date' column to datetime format with errors='coerce'
dt['Date'] = pd.to_datetime(dt['Date'], format='%Y.%m.%d', errors='coerce')
# Handle additional format
dt = dt.dropna(subset=['Date'])

In [21]:
dt['Home_Result'] = np.where(dt['home_score'] > dt['away_score'], 1, 0)

dt = dt[['Date', 'home_team', 'away_team', 'Home_Result']]

In [22]:
# Drop rows with missing values
dt = dt.dropna()

In [23]:
dt

,Date,home_team,away_team,Home_Result
0,1872-11-30,Scotland,England,0
1,1873-03-08,England,Scotland,1
2,1874-03-07,Scotland,England,1
3,1875-03-06,England,Scotland,0
4,1876-03-04,Scotland,England,1
...,...,...,...,...
45310,2023-11-21,Wales,Turkey,0
45311,2023-11-21,Croatia,Armenia,1
45312,2023-11-21,Andorra,Israel,0
45313,2023-11-21,Kosovo,Belarus,0


In [24]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45315 entries, 0 to 45314
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         45315 non-null  datetime64[ns]
 1   home_team    45315 non-null  object        
 2   away_team    45315 non-null  object        
 3   Home_Result  45315 non-null  int32         
dtypes: datetime64[ns](1), int32(1), object(2)
memory usage: 1.6+ MB


In [25]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pandas as pd

In [26]:
features = ['Date', 'home_team', 'away_team']
target = 'Home_Result'

# Split the data into features (X) and target variable (y)
X = dt[features]
y = dt[target]
Z= df[features]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [27]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


# Create transformers
date_transformer = Pipeline(steps=[
    ('date', None)  # No transformation for date columns
])

numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

# Define a categorical transformer
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Identify numerical and categorical columns
numerical_columns = ['Date']
categorical_columns = ['home_team', 'away_team']

# Create a preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_columns),
        ('cat', categorical_transformer, categorical_columns)
    ])


In [28]:
# Create a pipeline with preprocessing and the classifier
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression())
])

# Train the model
model.fit(X_train, y_train)

C:\Users\abben\AppData\Roaming\jupyterlab-desktop\jlab_server\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['Date']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['home_team',
                                                   'away_team'])])),
                ('classifier', LogisticRegression())])

In [29]:
# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)

# Print the evaluation metrics
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.68


## Appliquer votre modèle à la liste des matches créés dans l’exo 1 pour obtenir les résultats de tous les matchs de la phase de groupe de la CAN 2023.

In [31]:
# Make predictions
predictions = model.predict(Z)

# Add predictions to the DataFrame
df['Home_Result'] = predictions

## La Mauritanie se qualifiera-t-elle pour les 1/8 de finale?

In [34]:
mauritania_filter = (df['home_team'] == 'Mauritania') | (df['away_team'] == 'Mauritania')
mauritania_matches = df[mauritania_filter]

In [35]:
mauritania_matches

,Date,home_team,away_team,Home_Result
19,2024-01-16,Burkina Faso,Mauritania,1
21,2024-01-20,Mauritania,Angola,0


## Non, le modèle prédit que la Mauritanie perd dans les deux matches et ne se qualifiera pas pour les 1/8 de finale.

In [36]:
df

,Date,home_team,away_team,Home_Result
0,2024-01-13,Ivory Coast,Guinea Bissau,1
1,2024-01-14,Nigeria,Equatorial Guinea,1
2,2024-01-18,Equatorial Guinea,Guinea Bissau,0
3,2024-01-18,Ivory Coast,Nigeria,0
4,2024-01-22,Equatorial Guinea,Ivory Coast,0
6,2024-01-14,Egypt,Mozambique,1
7,2024-01-14,Ghana,Cape Verde,1
8,2024-01-18,Egypt,Ghana,1
9,2024-01-19,Cape Verde,Mozambique,1
10,2024-01-22,Cape Verde,Egypt,0
